<a href="https://colab.research.google.com/github/somaninggit/usplab/blob/main/Iot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Write a sample program to recognise port pins using getport.c

#include <stdio.h>
#include "NUC1xx.h"
#include "Driver\DrvSYS.h"
#include "Driver\DrvGPIO.h"
#include "NUC1xx-LB_002\LCD_Driver.h"
int main (void)
{
int32_t number;
char TEXT0[16]="SmplKeypad";
char TEXT1[16];
	UNLOCKREG();			    // unlock register for programming
  DrvSYS_Open(48000000);// set System Clock to run at 48MHz                     
	LOCKREG();				    // lock register from programming
        // Initialize LEDs (four on-board LEDs below LCD panel)	
Initial_panel(); 
	clr_all_panel();	    
	print_lcd(0,TEXT0); // print title	
	while (1)				   // forever loop to keep flashing four LEDs one at a time
	{
number=DrvGPIO_GetPortBits(E_GPA);			
	 sprintf(TEXT1,"%x",number); // print scankey input to string			  
	print_lcd(1,TEXT1); 
if(number==0xfffe)
print_lcd(2,"A0");	
else if(number==0xfffd)
	print_lcd(2,"A1");
else if(number==0xfffb)
	print_lcd(2,"A2");
else if(number==0xfff7)
	print_lcd(2,"A3");
else if(number==0xffef)
	print_lcd(2,"A4");
else if(number==0xffdf)
		print_lcd(2,"A5");
else if(number==0xffbf)
	print_lcd(2,"A6");
else if(number==0xff7f)
	print_lcd(2,"A7");
else if(number==0xfeff)
	print_lcd(2,"A8");
//else if(number==0xfeff)
	//print_lcd(2,"A7");
/*else if(number==0xff7f)
	print_lcd(2,"A7");
else if(number==0xff7f)
	print_lcd(2,"A7");
else if(number==0xff7f)
	print_lcd(2,"A7");
else if(number==0xff7f)
	print_lcd(2,"A7");
else if(number==0xff7f)
	print_lcd(2,"A7");
else if(number==0xff7f)
	print_lcd(2,"A7");
else if(number==0xff7f)
	print_lcd(2,"A7");
else if(number==0xff7f)
	print_lcd(2,"A7");*/


	}

}

In [ ]:
# 2. Write a program to use on  board interrupt INT1 and  external INT0 

#include <stdio.h>
#include "NUC1xx.h"
#include "Driver\DrvGPIO.h"

#include "Driver\DrvSYS.h"

// External Interrupt Handler (INT button to trigger GPB15/GPIB14)
void EINT1Callback(void) 
{
  DrvGPIO_ClrBit(E_GPB,11); // GPB11 = 0 to turn on Buzzer
	DrvSYS_Delay(1000000000000);	    // Delay 
	DrvGPIO_SetBit(E_GPB,11); // GPB11 = 1 to turn off Buzzer	
	DrvSYS_Delay(10000);	    // Delay 	
}

int main (void)
{
	UNLOCKREG();
	DrvSYS_SetOscCtrl(E_SYS_XTL12M, 1); // external 12MHz Crystal
	//DrvSYS_Delay(5000);                 // delay for stable clock
	DrvSYS_SelectHCLKSource(0);         // clock source = 12MHz Crystal
	LOCKREG();

	DrvGPIO_Open(E_GPB, 11, E_IO_OUTPUT); // initial GPIO pin GPB11 for controlling Buzzer
   
//0 External Interrupt 
  DrvGPIO_Open(E_GPB, 14, E_IO_INPUT);                             // configure external interrupt pin GPB15/GPIB14
  DrvGPIO_EnableEINT1(E_IO_BOTH_EDGE, E_MODE_EDGE, EINT1Callback); // configure external interrupt

  while(1)
	{
  }
}

In [ ]:
# 3. Write a program to change the intensity of led using PWM_LED.c  

#include <stdio.h>																											 
#include "NUC1xx.h"
#include "LCD_Driver.h"

void Delay(int count)
{
	while(count--)
	{
//		__NOP;
	 }
}

/*----------------------------------------------------------------------------
  MAIN function
  ----------------------------------------------------------------------------*/
int32_t main (void)
{
	//Enable 12Mhz and set HCLK->12Mhz
	char adc_value[15]="ADC Value:";	
	UNLOCKREG();
	SYSCLK->PWRCON.XTL12M_EN = 1;
	SYSCLK->CLKSEL0.HCLK_S = 0;
	LOCKREG();

	InitPWM();
	InitADC();

	Initial_panel();  //call initial pannel function
	clr_all_panel();
							 	
	/* Synch field transmission & Request Identifier Field transmission*/
					 
	while(1)
	{
		while(ADC->ADSR.ADF==0);
		ADC->ADSR.ADF=1;
		PWMA->CMR0=ADC->ADDR[7].RSLT<<4;
		Show_Word(0,11,' ');
		Show_Word(0,12,' ');
		Show_Word(0,13,' ');
		sprintf(adc_value+4,"%d",ADC->ADDR[7].RSLT);
		print_lcd(0, adc_value);
		Delay(20000);
		ADC->ADCR.ADST=1;
	}
}

In [ ]:
# 4. Write a sample program using setport.c using Interrupt 

#include <stdio.h>
#include "NUC1xx.h"
#include "Driver\DrvGPIO.h"

#include "Driver\DrvSYS.h"

void Init_LED() // Initialize GPIO pins
{
  DrvGPIO_Open(E_GPC, 12, E_IO_OUTPUT); // GPC12 pin set to output mode
	DrvGPIO_Open(E_GPC, 13, E_IO_OUTPUT);        // Goutput Hi to turn off LED
	DrvGPIO_Open(E_GPC, 14, E_IO_OUTPUT);
	DrvGPIO_Open(E_GPC, 15, E_IO_OUTPUT);
}
void call()
{
DrvGPIO_SetPortBits(E_GPC,0xffff0fff); // output Low to turn on LED
	DrvSYS_Delay(300000);	   // delay 
DrvGPIO_SetPortBits(E_GPC,0xffffffff)	; // output Hi to turn off LED
	DrvSYS_Delay(300000);	   // delay
}
int main (void)
{
	UNLOCKREG();			    // unlock register for programming
  DrvSYS_Open(48000000);// set System Clock to run at 48MHz 
	                      // 12MHz crystal input, PLL output 48MHz
	LOCKREG();				    // lock register from programming

  Init_LED();        // Initialize LEDs (four on-board LEDs below LCD panel)
  DrvGPIO_Open(E_GPB, 15, E_IO_INPUT);                             // configure external interrupt pin GPB15
  DrvGPIO_EnableEINT1(E_IO_BOTH_EDGE, E_MODE_EDGE, call); // configure external interrupt

	while (1)				   // forever loop to keep flashing four LEDs one at a time
	{
	
	}

}


In [ ]:
# 5.	 Write a program to use adc with functions.c choosing channel 6

#include <stdio.h>																											 
#include "NUC1xx.h"
#include "Driver\DrvSYS.h"
#include "Seven_Segment.h"
#include "DrvADC.h"
#include "LCD_Driver.h"

	 
 int32_t main (void)
{  uint16_t value;  
    char TEXT[16];			   
	
	UNLOCKREG();
	SYSCLK->PWRCON.XTL12M_EN = 1; 	//Enable 12Mhz and set HCLK->12Mhz
	SYSCLK->CLKSEL0.HCLK_S = 0;
	LOCKREG();
	Initial_panel();  // initialize LCD pannel
  clr_all_panel();  // clear LCD panel 
  print_lcd(0,"variable reistor");

	DrvADC_Open(ADC_SINGLE_END,ADC_SINGLE_OP , 0x40,INTERNAL_HCLK , 1); 	
	while(1)
   {
    DrvADC_StartConvert();   // start A/D conversion
    while(DrvADC_IsConversionDone()==FALSE);
    value = ADC->ADDR[6].RSLT & 0xFFF;    
    
    sprintf(TEXT,"Value: %d",value); // convert ADC0 value into text
    print_lcd(1, TEXT); // output TEXT to LCD
   
    }
}				 	


In [ ]:
# 6.	 Write a program to use Smpl_LCD_Text.c using Interrupt

#include <stdio.h>
#include "NUC1xx.h"
#include "Driver\DrvSYS.h"
#include "Driver\DrvGPIO.h"
#include "NUC1xx-LB_002\LCD_Driver.h"
void call()
{
	print_lcd(0, "Smpl_LCD_Text   ");	  
	print_lcd(1, "Nu-LB-NUC140    ");
	print_lcd(2, "Test LCD Display");
	print_lcd(3, "Nuvoton NuMicro ");  
}
int main(void)
{
	UNLOCKREG();
	DrvSYS_Open(48000000); // set to 48MHz
	LOCKREG(); 

	Initial_panel(); 
	clr_all_panel();
	
	 DrvGPIO_Open(E_GPB, 15, E_IO_INPUT);                             // configure external interrupt pin GPB15
  DrvGPIO_EnableEINT1(E_IO_BOTH_EDGE, E_MODE_EDGE, call); // configure external interrupt

  while(1)
	{
  }
}  		


In [ ]:
# 7.	 Smpl_GPIO_Buzzer.c using Interrupt

#include <stdio.h>
#include "NUC1xx.h"
#include "Driver\DrvGPIO.h"

#include "Driver\DrvSYS.h"

// External Interrupt Handler (INT button to trigger GPB15)
void EINT1Callback(void) 
{
  DrvGPIO_ClrBit(E_GPB,11); // GPB11 = 0 to turn on Buzzer
	DrvSYS_Delay(1000000000000);	    // Delay 
	DrvGPIO_SetBit(E_GPB,11); // GPB11 = 1 to turn off Buzzer	
	DrvSYS_Delay(10000);	    // Delay 	
}

int main (void)
{
	UNLOCKREG();
	DrvSYS_SetOscCtrl(E_SYS_XTL12M, 1); // external 12MHz Crystal
	//DrvSYS_Delay(5000);                 // delay for stable clock
	DrvSYS_SelectHCLKSource(0);         // clock source = 12MHz Crystal
	LOCKREG();

	DrvGPIO_Open(E_GPB, 11, E_IO_OUTPUT); // initial GPIO pin GPB11 for controlling Buzzer
   
//0 External Interrupt 
  DrvGPIO_Open(E_GPB, 15, E_IO_INPUT);                             // configure external interrupt pin GPB15
  DrvGPIO_EnableEINT1(E_IO_BOTH_EDGE, E_MODE_EDGE, EINT1Callback); // configure external interrupt

  while(1)
	{
  }
}

In [ ]:
# 8.	 Smpl_GPIO_Interrupt.c using port pins of Port A and Port E

#include <stdio.h>
#include "NUC1xx.h"
#include "Driver\DrvUART.h"
#include "Driver\DrvGPIO.h"
#include "Driver\DrvSYS.h"
#include "LCD_Driver.h"

volatile uint32_t irqA_counter = 0;
volatile uint32_t irqE_counter = 0;

void GPIOAB_INT_CallBack(uint32_t GPA_IntStatus, uint32_t GPB_IntStatus)
{
	 if ((GPA_IntStatus>>15) & 0x01) irqA_counter++;
	 print_lcd(3,"GPA interrupt !!");
}

void GPIOCDE_INT_CallBack(uint32_t GPC_IntStatus, uint32_t GPD_IntStatus, uint32_t GPE_IntStatus)
{  
	 if ((GPE_IntStatus>>15) & 0x01) irqE_counter++;
	 print_lcd(3,"GPC interrupt !!");
}

int32_t main()
{
	char TEXT[16];

	UNLOCKREG();
	SYSCLK->PWRCON.XTL12M_EN=1;
	DrvSYS_Delay(5000);					// Waiting for 12M Xtal stalble
	SYSCLK->CLKSEL0.HCLK_S=0;
	LOCKREG();

  // setup GPA15 & GPD15 to get interrupt input
	DrvGPIO_Open(E_GPA,15,E_IO_INPUT);
	DrvGPIO_Open(E_GPE,15,E_IO_INPUT);
  DrvGPIO_EnableInt(E_GPA, 15, E_IO_RISING, E_MODE_EDGE);
  DrvGPIO_EnableInt(E_GPE, 15, E_IO_RISING, E_MODE_EDGE);
  DrvGPIO_SetDebounceTime(5, 1);
	DrvGPIO_EnableDebounce(E_GPA, 15);
	DrvGPIO_EnableDebounce(E_GPE, 15);	
	
  DrvGPIO_SetIntCallback(GPIOAB_INT_CallBack, GPIOCDE_INT_CallBack);

  Initial_panel();
	clr_all_panel();
    				
	print_lcd(0,"Smpl_GPIO_Intr");
	
	while(1)
	{
		sprintf(TEXT,"IRQ_A: %d",irqA_counter);
		print_lcd(1, TEXT);
		sprintf(TEXT,"IRQ_E: %d",irqE_counter);
		print_lcd(2, TEXT);
	}
}


In [ ]:
# 9.	 Smpl_GPIO_LED1.c using Interrupt INT0 or INT1 

#include <stdio.h>
#include "NUC1xx.h"
#include "Driver\DrvGPIO.h"

#include "Driver\DrvSYS.h"
void call()
{
	DrvGPIO_ClrBit(E_GPC, 12); // output Low to turn on LED
	DrvSYS_Delay(300000);	   // delay 
	DrvGPIO_SetBit(E_GPC, 12); // output Hi to turn off LED
	DrvSYS_Delay(300000);	   // delay
}
void Init_LED() // Initialize GPIO pins
{
 DrvGPIO_Open(E_GPC, 12, E_IO_OUTPUT); // GPC12 pin set to output mode
	DrvGPIO_SetBit(E_GPC, 12);            // Goutput Hi to turn off LED
}

int main (void)
{
	UNLOCKREG();			    // unlock register for programming
  DrvSYS_Open(48000000);// set System Clock to run at 48MHz 
	                      // 12MHz crystal input, PLL output 48MHz
	LOCKREG();				    // lock register from programming

  Init_LED();        // Initialize LEDs (four on-board LEDs below LCD panel)
   DrvGPIO_Open(E_GPB, 15, E_IO_INPUT);                             // configure external interrupt pin GPB15
  DrvGPIO_EnableEINT1(E_IO_BOTH_EDGE, E_MODE_EDGE,call); // configure external interrupt
	while (1)				   // forever loop to keep flashing four LEDs one at a time
	{
  // delay
	}

}


In [ ]:
# 10.	 Smpl_GPIO_RGBled.c  using Interrupt

#include <stdio.h>
#include "NUC1xx.h"
#include "Driver\DrvGPIO.h"
#include "Driver\DrvUART.h"
#include "Driver\DrvSYS.h"

// External Interrupt Handler (INT button to trigger GPB15)
void EINT1Callback(void) 
{
 //DrvGPIO_ClrBit(E_GPB,11); // GPB11 = 0 to turn on Buzzer
	//DrvSYS_Delay(10);	    // Delay 
		 
	// GPA12 = Blue,  0 : on, 1 : off
	// GPA13 = Green, 0 : on, 1 : off
	// GPA14 = Red,   0 : on, 1 : off

	// set RGBled to Blue
    DrvGPIO_ClrBit(E_GPA,12); // GPA12 = Blue,  0 : on, 1 : off
    DrvGPIO_SetBit(E_GPA,13); 
    DrvGPIO_SetBit(E_GPA,14); 
	DrvSYS_Delay(1000000); 		   

	// set RGBled to Green
    DrvGPIO_SetBit(E_GPA,12); 
    DrvGPIO_ClrBit(E_GPA,13); // GPA13 = Green, 0 : on, 1 : off
    DrvGPIO_SetBit(E_GPA,14); 
	DrvSYS_Delay(1000000);			  

	// set RGBled to Red
    DrvGPIO_SetBit(E_GPA,12); 
    DrvGPIO_SetBit(E_GPA,13); 
    DrvGPIO_ClrBit(E_GPA,14); // GPA14 = Red,   0 : on, 1 : off
	DrvSYS_Delay(1000000);			  
	
	// set RGBled to off	 
    DrvGPIO_SetBit(E_GPA,12); // GPA12 = Blue,  0 : on, 1 : off
    DrvGPIO_SetBit(E_GPA,13); // GPA13 = Green, 0 : on, 1 : off
    DrvGPIO_SetBit(E_GPA,14); // GPA14 = Red,   0 : on, 1 : off
	DrvSYS_Delay(1000000);

	 // GPB11 = 1 to turn off Buzzer	
	//DrvSYS_Delay(10000);	    // Delay 	
}
void Init_LED()
{
	// initialize GPIO pins
	DrvGPIO_Open(E_GPA, 12, E_IO_OUTPUT); // GPA12 pin set to output mode
	DrvGPIO_Open(E_GPA, 13, E_IO_OUTPUT); // GPA13 pin set to output mode
	DrvGPIO_Open(E_GPA, 14, E_IO_OUTPUT); // GPA14 pin set to output mode
	// set GPIO pins output Hi to disable LEDs
	DrvGPIO_SetBit(E_GPA, 12); // GPA12 pin output Hi to turn off Blue  LED
	DrvGPIO_SetBit(E_GPA, 13); // GPA13 pin output Hi to turn off Green LED
	DrvGPIO_SetBit(E_GPA, 14); // GPA14 pin output Hi to turn off Red   LED
}  
int main (void)
{
	UNLOCKREG();
	DrvSYS_SetOscCtrl(E_SYS_XTL12M, 1); // external 12MHz Crystal
	//DrvSYS_Delay(5000);                 // delay for stable clock
	DrvSYS_SelectHCLKSource(0);         // clock source = 12MHz Crystal
	LOCKREG();
	Init_LED();
	DrvGPIO_Open(E_GPB, 11, E_IO_OUTPUT); // initial GPIO pin GPB11 for controlling Buzzer
   
//0 External Interrupt 
  DrvGPIO_Open(E_GPB, 15, E_IO_INPUT);                             // configure external interrupt pin GPB15
  DrvGPIO_EnableEINT1(E_IO_BOTH_EDGE, E_MODE_EDGE, EINT1Callback); // configure external interrupt

  while(1)
	{
  }
}


In [ ]:
# 11.	 Write a program to display 0 t0 9999 using 7 segment

#include <stdio.h>																											 
#include "NUC1xx.h"
#include "DrvSYS.h"
#include "Seven_Segment.h"

// display an integer on four 7-segment LEDs
void seg_display(int16_t value)
{
  int8_t digit;
		digit = value / 1000;
		close_seven_segment();
		show_seven_segment(3,digit);
		DrvSYS_Delay(5000);
			
		value = value - digit * 1000;
		digit = value / 100;
		close_seven_segment();
		show_seven_segment(2,digit);
		DrvSYS_Delay(5000);

		value = value - digit * 100;
		digit = value / 10;
		close_seven_segment();
		show_seven_segment(1,digit);
		DrvSYS_Delay(5000);

		value = value - digit * 10;
		digit = value;
		close_seven_segment();
		show_seven_segment(0,digit);
		DrvSYS_Delay(5000);
}

int32_t main (void)
{
	int32_t i =0;
	
	UNLOCKREG();
	DrvSYS_Open(48000000);
	LOCKREG();			 	

 	while(i<10000) 
	{
	   seg_display(i);		// display i on 7-segment display
	   DrvSYS_Delay(10000);	// delay for keeping display
	   i++;					// increment i
	}
}


In [ ]:
# 12.	 Using SSH blink led using rasberry Pi from remote system 

from gpiozero import LED
from signal import pause

led=LED(4)
led.blink()

pause()

In [ ]:
# 13.	Using SSH motion detector using rasberry Pi from remote system 

from gpiozero import MotionSensor
from signal import pause

motion_sensor = MotionSensor(4)

def motion():
  print("Motion detected")

def no_motion():
  print("Motion stopped")

print("Readying sensor...")
motion_sensor.wait_for_no_motion()
print("Sensor ready")

motion_sensor.when_motion = motion
motion_sensor.when_no_motion = no_motion

pause()

In [ ]:
# 14.	Using SSH key pressed using rasberry Pi from remote system 

from gpiozero import Button
from signal import pause

button = Button(4)

def button_pressed():
  print("Button was pressed")

def button_held():
  print("Button was held")

def button_released():
  print("Button was released")

button.when_pressed = button_pressed
button.when_held = button_held
button.when_released = button_released

pause()
